In [1]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


# **PyTorch Linear regression tp predict fish weight** 

In [2]:
import torch
from sklearn import preprocessing

In [3]:
df = pd.read_csv('Fish.csv')

In [4]:
df.shape

(159, 7)

# Check fish species 

In [5]:
df['Species'].value_counts()

Perch        56
Bream        35
Roach        20
Pike         17
Smelt        14
Parkki       11
Whitefish     6
Name: Species, dtype: int64

In [6]:
le = preprocessing.LabelEncoder()


## Convert categorical data to integer 

In [7]:
le.fit(['Perch', 'Bream', 'Roach', 'Pike', 'Smelt', 'Parkki','Whitefish' ])

LabelEncoder()

In [8]:
sp = df['Species']
sp[0]

'Bream'

In [9]:
list(le.classes_)


['Bream', 'Parkki', 'Perch', 'Pike', 'Roach', 'Smelt', 'Whitefish']

In [10]:
Species = le.transform(sp)

In [11]:
sp = Species.tolist()

In [12]:
df = df.drop('Species', axis=1)
scaler = preprocessing.StandardScaler().fit(df)
scaler.transform(df)

array([[-4.38072172e-01, -3.05788578e-01, -2.82303007e-01,
        -1.06020232e-01,  5.96578670e-01, -2.36528948e-01],
       [-3.03562184e-01, -2.25507242e-01, -1.98053663e-01,
        -2.33668373e-03,  8.21260549e-01, -6.65789457e-02],
       [-1.63447613e-01, -2.35542409e-01, -1.79331587e-01,
        -1.09769794e-02,  7.97341291e-01,  1.65793169e-01],
       [-9.89949100e-02,  5.30159764e-03,  5.46943678e-02,
         1.96390116e-01,  8.79771455e-01,  2.26210031e-02],
       [ 8.87586153e-02,  2.53719316e-02,  5.46943678e-02,
         2.39591594e-01,  8.12834979e-01,  4.26371272e-01],
       [ 1.44804444e-01,  5.54774324e-02,  1.20221635e-01,
         3.00073664e-01,  1.08395111e+00,  3.03431249e-01],
       [ 2.84919015e-01,  5.54774324e-02,  1.20221635e-01,
         2.82793073e-01,  1.21901769e+00,  5.12357880e-01],
       [-2.33330416e-02,  1.35758768e-01,  1.48304750e-01,
         3.25994551e-01,  8.65728838e-01,  1.62163285e-01],
       [ 1.44804444e-01,  1.35758768e-01,  1.483

In [13]:
df.insert(0, 'sp', Species)

In [14]:
df.head()


,sp,Weight,Length1,Length2,Length3,Height,Width
0,0,242.0,23.2,25.4,30.0,11.5200,4.0200
1,0,290.0,24.0,26.3,31.2,12.4800,4.3056
2,0,340.0,23.9,26.5,31.1,12.3778,4.6961
3,0,363.0,26.3,29.0,33.5,12.7300,4.4555
4,0,430.0,26.5,29.0,34.0,12.4440,5.1340


## Create labels and dataset 

In [15]:
labels = df['Weight']
dataset = df.drop('Weight',axis=1)

## Shuffel the data

In [16]:
from sklearn.utils import shuffle
dataset, labels = shuffle(dataset,labels)

In [17]:
dataset.shape, labels.shape


((159, 6), (159,))

## Convert to Tensor 

In [18]:
labels = torch.tensor(labels.values.astype(np.float32))
dataset = torch.tensor(dataset.values.astype(np.float32))
labels = labels.unsqueeze(1)

In [19]:
dataset.shape, labels.shape

(torch.Size([159, 6]), torch.Size([159, 1]))

In [20]:
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset

In [21]:
train_d = TensorDataset(dataset, labels)
train_d[0:4]

(tensor([[ 1.0000, 14.3000, 15.5000, 17.4000,  6.5772,  2.3142],
         [ 2.0000, 25.2000, 27.3000, 28.7000,  8.3230,  5.1373],
         [ 4.0000, 21.1000, 22.5000, 25.0000,  6.4000,  3.8000],
         [ 2.0000, 30.5000, 32.8000, 34.0000, 10.0300,  6.0180]]),
 tensor([[ 60.],
         [300.],
         [160.],
         [514.]]))

In [22]:
batch_size = 300
train_dl = DataLoader(train_d, batch_size, shuffle=True)

## Create the model 

In [23]:
import torch.nn as nn

In [24]:
model = nn.Linear(6, 1)


In [25]:
list(model.parameters())

[Parameter containing:
 tensor([[ 0.0340, -0.0032,  0.2382, -0.0377, -0.4034, -0.2200]],
        requires_grad=True),
 Parameter containing:
 tensor([0.2734], requires_grad=True)]

In [26]:
preds = model(dataset)

In [27]:
import torch.nn.functional as F

In [28]:
loss_fn = F.mse_loss

In [29]:
loss = loss_fn(model(dataset), labels)
loss

tensor(284779.5000, grad_fn=<MseLossBackward>)

In [30]:
opt = torch.optim.SGD(model.parameters(), lr=0.0002) 

## Create the training loop

In [31]:
def fit(num_epoches, model, loss_fn, opt):
    for epoch in range(num_epoches):
        for xb, yb in train_d:
            pred = model(xb)
            loss = loss_fn(pred, yb)
            loss.backward()
            opt.step()
            opt.zero_grad()
            
        if (epoch+1) % 10== 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epoches, loss.item()))
        if loss.item() < 5 and loss.item() > 0:
            break

In [35]:
fit(300, model, loss_fn, opt)

Epoch [10/300], Loss: 1195.9655
Epoch [20/300], Loss: 1077.9122
Epoch [30/300], Loss: 970.0940
Epoch [40/300], Loss: 871.6604
Epoch [50/300], Loss: 781.8789
Epoch [60/300], Loss: 700.0327
Epoch [70/300], Loss: 625.5174
Epoch [80/300], Loss: 557.7254
Epoch [90/300], Loss: 496.1033
Epoch [100/300], Loss: 440.1826
Epoch [110/300], Loss: 389.4841
Epoch [120/300], Loss: 343.5860
Epoch [130/300], Loss: 302.0858
Epoch [140/300], Loss: 264.6440
Epoch [150/300], Loss: 230.9031
Epoch [160/300], Loss: 200.5757
Epoch [170/300], Loss: 173.3713
Epoch [180/300], Loss: 149.0295
Epoch [190/300], Loss: 127.3052
Epoch [200/300], Loss: 107.9943
Epoch [210/300], Loss: 90.8773
Epoch [220/300], Loss: 75.7688
Epoch [230/300], Loss: 62.5088
Epoch [240/300], Loss: 50.9310
Epoch [250/300], Loss: 40.8838
Epoch [260/300], Loss: 32.2422
Epoch [270/300], Loss: 24.8805
Epoch [280/300], Loss: 18.6788
Epoch [290/300], Loss: 13.5356
Epoch [300/300], Loss: 9.3576


In [38]:
preds = model(dataset)
preds[:15]

tensor([[  -9.1694],
        [ 323.9931],
        [ 313.0934],
        [ 523.1842],
        [  78.3380],
        [ 405.4038],
        [ 362.3824],
        [ 216.1292],
        [ 535.1688],
        [ 232.2432],
        [1073.2478],
        [ 510.4905],
        [ 692.2057],
        [  53.3680],
        [ 281.0114]], grad_fn=<SliceBackward>)

In [39]:
print(labels[:15])

tensor([[  60.],
        [ 300.],
        [ 160.],
        [ 514.],
        [  85.],
        [ 300.],
        [ 300.],
        [ 197.],
        [ 725.],
        [ 340.],
        [1550.],
        [ 575.],
        [ 650.],
        [  85.],
        [   0.]])


In [42]:
!pip install jovian

  Using cached requests-2.23.0-py2.py3-none-any.whl (58 kB)
  Using cached uuid-1.30.tar.gz (5.8 kB)
  Using cached idna-2.9-py2.py3-none-any.whl (58 kB)
  Using cached chardet-3.0.4-py2.py3-none-any.whl (133 kB)
  Created wheel for uuid: filename=uuid-1.30-py3-none-any.whl size=6507 sha256=00f67bc2a1afe05a09154be9da5bd98ee1415ed491dc16be0a902555c486525b
  Stored in directory: c:\users\mosta\appdata\local\pip\cache\wheels\20\18\ca\566a05e0cb4865f6a264a809b207181ef5cc5f32b26c4c41c6
Successfully built uuid


In [45]:
import jovian

In [47]:
jovian.commit()

<IPython.core.display.Javascript object>

[jovian] Attempting to save notebook..
[jovian] Updating notebook "mustafaazzurr/kernel671d9b3f42" on https://jovian.ml/
[jovian] Uploading notebook..
[jovian] Capturing environment..
[jovian] Committed successfully! https://jovian.ml/mustafaazzurr/kernel671d9b3f42


'https://jovian.ml/mustafaazzurr/kernel671d9b3f42'